In [101]:
import scipy.io
import numpy
import scipy.signal

reads_file = 'filteredData_test.mat'
TF_motif_file = 'filteredPeakAsso_test.mat'
Transposase_bias_file = 'bgParem_test.mat'
#
# read the orginal reads data, TF motif data, Tn5 bias data
reads = scipy.io.loadmat(reads_file)['filteredData']   # as filteredData
TFmotif = scipy.io.loadmat(TF_motif_file)['filteredPeakAssoc'].transpose()   # as filteredPeakAssoc
bias = scipy.io.loadmat(Transposase_bias_file)['bgParem'].transpose()[0]   # as bgParem
(nCell, nPeak) = reads.shape
nTF = TFmotif.shape[0]
#
nIteration = 6
conv_kernel_size = 4  # as kernelSize
motif_per_TF = TFmotif.sum(axis = 1)  # as totAssoc
reads_per_cell = reads.sum(axis = 1)  # as readsPerCell
reads_per_peak = reads.sum(axis = 0)  # as readsPerRegion and readSort
total_reads = reads.sum()
#
Ipeak = numpy.argsort(reads_per_peak)  # as Ipeak
bias_Ipeak = bias[Ipeak]  # as Bpeak[2, :]
Ibias = numpy.argsort(bias)  # as Ibias
readsPerPeak_Ibias = reads_per_peak[Ibias]  # as Bbias[1, :]
Ipeak_rev = numpy.argsort(Ipeak)  # reads_per_peak[Ipeak[Ipeak_rev]] = reads_per_peak
Ibias_rev = numpy.argsort(Ibias)  # bias[Ibias[Ibias_rev]] = bias
#
sample_perTFperPeak = numpy.zeros((nIteration, nPeak, nTF))
sample_perTFperCell = numpy.zeros((nTF, nCell))
sample_matrix = numpy.zeros((nIteration, nTF, nCell))
#
def weights(TFmotif, Index, conv_kernel_size):
    TFmotif_Index = TFmotif[:, Index]  # as filteredPeakAssocSort
    extend_TFmotif = numpy.c_[TFmotif_Index[:, (conv_kernel_size / 2 - 1)::-1], TFmotif_Index, 
                              TFmotif_Index[:, -1:(-conv_kernel_size / 2 - 1):-1]]  # as paddedPAS
    extend_TFmotif = extend_TFmotif[:, 0:-1]
    conv_window = numpy.ones((1, conv_kernel_size))
    smooth_TFmotif_Index = scipy.signal.convolve(extend_TFmotif, conv_window, 
                                                 mode='valid') / conv_kernel_size
    return smooth_TFmotif_Index
#
wPeak = weights(TFmotif, Ipeak, conv_kernel_size)
wBias = weights(TFmotif, Ibias, conv_kernel_size)
wBias_revIbias = wBias[:, Ibias_rev]
wPeakBias = weights(wBias_revIbias, Ipeak, conv_kernel_size)
# 
for iIteration in range(0, nIteration):
    for iTF in range(0, nTF):
        weight = wPeak[iTF, :] / wPeak[iTF, :].sum()
        index_sample = numpy.random.choice(xrange(nPeak), size=motif_per_TF[iTF], p=weight)
        print iIteration, iTF
        print index_sample

0 0
[16 16 15  2  5  3  5 18  2  0  2 18 18 13]
0 1
[17  2 12  1  5 12 14 12  4 12 14  4 11 12]
0 2
[12  2 11  3 11  3  6 13  7  0 15]
0 3
[ 2  1  5  7  9 12  7 10]
0 4
[ 5  7 14 15 14  1]
0 5
[11  7  5 18 14  7 19  8]
0 6
[ 2 18 14 18 12  5 15 15 12 16]
1 0
[ 0 17 11 19 12  2  1  8 17 14  1 17 14  6]
1 1
[10  0  0 19 13  0 17 12  3  6  4 10  3 10]
1 2
[ 6 14  2 10  3 14 16 16  5 15 19]
1 3
[ 5  6  4  3  0 16 12  8]
1 4
[17 12 16  2 14  8]
1 5
[12  7  7  5 10 15  8 18]
1 6
[15 16 14 13  7 18  9 14 18 13]
2 0
[ 5  0 13 19  0 16  0  7  2 16 13 17  7 16]
2 1
[ 2 13  7 16 15  6 10  7 15 17 15 17 10 13]
2 2
[10 15 15  4  3 11 15 16  2  7  9]
2 3
[ 7  1 16 14  4 15 10 16]
2 4
[15 12 13 14 17 18]
2 5
[15  5 15 18  8 16  7  5]
2 6
[15 12  5 13 12 12 14  8 14 12]
3 0
[16  4 10  4 16  0  5  4 17 11  0  6 11  4]
3 1
[10  4 15  2 17  6 11 12 11 15  2  2 11 18]
3 2
[ 6 13 12 15  9  2 14 15 11  9  3]
3 3
[ 0 10 18 16  0  5 13 16]
3 4
[ 8 19 12  8 16  5]
3 5
[ 6  5  5 16 13 10  9 18]
3 6
[ 5  4  6 11